## 生成FDA Drug Label的关系数据

In [2]:
import pandas as pd
import os
import json
import re
os.getcwd()

'D:\\pgkb_graph'

In [7]:
df_drug_label = pd.read_csv('drug_label/drugLabels.tsv', sep='\t').fillna("")
# df_drug_label = df_drug_label[df_drug_label["Source"] == "FDA"]
print(df_drug_label.shape)
df_drug_label[:10]

(801, 14)


,PharmGKB ID,Name,Source,Biomarker Flag,Testing Level,Has Prescribing Info,Has Dosing Info,Has Alternate Drug,Cancer Genome,Prescribing,Chemicals,Genes,Variants/Haplotypes,Latest History Date (YYYY-MM-DD)
0,PA166184405,Annotation of Swissmedic Label for brexpiprazo...,Swissmedic,,Actionable PGx,Prescribing Info,Dosing Info,,,Prescribing,brexpiprazole,CYP2D6,,
1,PA166184114,Annotation of Swissmedic Label for capecitabin...,Swissmedic,,Actionable PGx,Prescribing Info,Dosing Info,Alternate Drug,,Prescribing,capecitabine,DPYD,,
2,PA166184116,Annotation of Swissmedic Label for carbamazepi...,Swissmedic,,Testing required,Prescribing Info,,Alternate Drug,,Prescribing,carbamazepine,HLA-B,HLA-B*15:02:01,
3,PA166184117,Annotation of Swissmedic Label for carbasalate...,Swissmedic,,Actionable PGx,,,,,,carbasalate calcium,G6PD,,
4,PA166184122,Annotation of Swissmedic Label for ciprofloxac...,Swissmedic,,Actionable PGx,,,,,,ciprofloxacin,G6PD,,
5,PA166184124,Annotation of Swissmedic Label for clomipramin...,Swissmedic,,Actionable PGx,,,,,,clomipramine,CYP2D6,,
6,PA166184126,Annotation of Swissmedic Label for codeine and...,Swissmedic,,Actionable PGx,Prescribing Info,,Alternate Drug,,Prescribing,codeine,CYP2D6,,
7,PA166184133,Annotation of Swissmedic Label for dexlansopra...,Swissmedic,,Actionable PGx,,,,,,dexlansoprazole,CYP2C19,,
8,PA166184134,Annotation of Swissmedic Label for dextrometho...,Swissmedic,,Actionable PGx,,,,,,dextromethorphan,CYP2D6,,
9,PA166184506,Annotation of Swissmedic Label for dolutegravi...,Swissmedic,,Testing required,Prescribing Info,,,,Prescribing,dolutegravir / abacavir / lamivudine,HLA-B,HLA-B*57:01,


In [8]:
drug_gene_label_list = []
drug_variant_label_list = []

for index, row in df_drug_label.iterrows():
    genes = row["Genes"]
    gene_list = [x.strip() for x in genes.split(";")]
    variant = row["Variants/Haplotypes"]
    variant_list = filter(lambda x: x!= "", [x.strip() for x in variant.split(";")])
    chemicals = row["Chemicals"]
    chemical_list = filter(lambda x: x!= "", [x.strip() for x in re.split(r"and|/", chemicals)])
    source = row["Source"]
    level = row["Testing Level"]
    name = row["Name"]
    update_date = row["Latest History Date (YYYY-MM-DD)"]
    
    
    for c in chemical_list:
        for g in gene_list:
            drug_gene_label_list.append((c, level, source, name, update_date, g))
            
        for v in variant_list:
            drug_variant_label_list.append((c, level, source, name, update_date, v))


In [9]:
print(len(drug_variant_label_list))
drug_variant_label_list[:2]

187


[('carbamazepine',
  'Testing required',
  'Swissmedic',
  'Annotation of Swissmedic Label for carbamazepine and HLA-B',
  '',
  'HLA-B*15:02:01'),
 ('dolutegravir',
  'Testing required',
  'Swissmedic',
  'Annotation of Swissmedic Label for dolutegravir / abacavir / lamivudine and HLA-B',
  '',
  'HLA-B*57:01')]

In [10]:
print(len(drug_gene_label_list))
drug_gene_label_list[:2]

1274


[('brexpiprazole',
  'Actionable PGx',
  'Swissmedic',
  'Annotation of Swissmedic Label for brexpiprazole and CYP2D6',
  '',
  'CYP2D6'),
 ('capecitabine',
  'Actionable PGx',
  'Swissmedic',
  'Annotation of Swissmedic Label for capecitabine and DPYD',
  '',
  'DPYD')]

In [11]:
pd.DataFrame(drug_gene_label_list, 
             columns=["drug", "label", "organization", "name", "update_date", "gene"]).assign(
    data_source=["drug_label"] * len(drug_gene_label_list)).to_csv("processed/drug_gene_label.csv", index=False)

pd.DataFrame(drug_variant_label_list, 
             columns=["drug", "label", "organization", "name", "update_date", "variant"]).assign(
    data_source=["drug_label"] * len(drug_variant_label_list)).to_csv("processed/drug_variant_label.csv", index=False)